# Solution Example

This notebook contains a solution example. Only use this if time is about to run out and you cannot find a solution on your own.

In [ ]:
# import libraries
import time
import grovepi
import math
import psycopg2
import time
from datetime import datetime

You need to fill in the **password and Database Name** to build up the connection!

The Database Name is iot_db_*XX* (e.g. Group 1: iot_db_01)

In [ ]:
# open connection to database
conn = psycopg2.connect(
    dbname='', # FILL IN THE DATABASE NAME OF YOUR GROUP
    user='nimda@iot-zeiss-2504', 
    host='iot-zeiss-2504.postgres.database.azure.com', 
    password='', # FILL IN THE DATABASE PASSWORD
    port='5432', 
    sslmode='require')
cursor = conn.cursor()
print("Connection established")

# delete table
cursor.execute("DROP TABLE IF EXISTS project_solution;")
print("Finished dropping table (if existed)")

# create table
cursor.execute("""
CREATE TABLE project_solution (
    temperature_column FLOAT8, 
    humidity_column FLOAT8, 
    light_column FLOAT8, 
    sound_column FLOAT8, 
    ultrasonic_ranger_column FLOAT8, 
    date DATE, 
    time TIME);""")
print("Finished creating table")

#close connection to database
conn.commit()
cursor.close()
conn.close()
print("connection closed")

You need to fill in the **password and Database Name** to build up the connection!

The Database Name is iot_db_*XX* (e.g. Group 1: iot_db_01)

In [ ]:
# open connection to database
conn = psycopg2.connect(
    dbname='', # FILL IN THE DATABASE NAME OF YOUR GROUP
    user='nimda@iot-zeiss-2504', 
    host='iot-zeiss-2504.postgres.database.azure.com', 
    password='', # FILL IN THE DATABASE PASSWORD
    port='5432', 
    sslmode='require')
cursor = conn.cursor()
print("Connection established")

# Sensor connections:
temp_humidity_sensor = 4  # The Sensor goes on digital port D4.
light_sensor = 1 # Connect the Grove Light Sensor to analog port A1
sound_sensor = 0 # Connect the Grove Sound Sensor to analog port A0
ultrasonic_ranger = 5 # Connect the Grove Ultrasonic Ranger to digital port D5

# Define Pin Mode
grovepi.pinMode(temp_humidity_sensor,"INPUT")
grovepi.pinMode(light_sensor,"INPUT")
grovepi.pinMode(sound_sensor,"INPUT")
grovepi.pinMode(ultrasonic_ranger,"INPUT")

# loop to collect data
while True:
    try:      
        # Get sensor value 
        [temp_value,humidity_value] = grovepi.dht(temp_humidity_sensor,0)
        light_value = grovepi.analogRead(light_sensor)
        sound_value = grovepi.analogRead(sound_sensor)
        ultrasonic_sensor_value = grovepi.ultrasonicRead(ultrasonic_ranger)

        # set date and time
        date_value = datetime.today().strftime("%Y-%m-%d")
        time_value = datetime.today().strftime("%H:%M:%S")
        
        # print the values that the sensor collected
        print(f"Temperatur = {temp_value}C \t Humidity = {humidity_value}% \t Light = {light_value} \t Sound = {sound_value} \t Ultrasonic Ranger = {ultrasonic_sensor_value}")

        # insert data into your project database
        cursor.execute("""
        INSERT INTO project_solution (
            temperature_column, 
            humidity_column, 
            light_column, 
            sound_column, 
            ultrasonic_ranger_column, 
            date, 
            time) 
        VALUES (%s, %s, %s, %s, %s, %s, %s);""", (temp_value, humidity_value, light_value, sound_value, ultrasonic_sensor_value, date_value, time_value))
        
        # wait 1 second        
        time.sleep(1)       

    except IOError:
        print ("Error")

    except KeyboardInterrupt:
        # close connection
        conn.commit()
        cursor.close()
        conn.close()
        print("connection closed")
        break